# Machine Learning
## Assignment 2

# Group 4

# Naive Bayes for Spam

In [1]:
import pandas as pd
import numpy as np

### 1. Load data

In [2]:
df = pd.read_table("SMSSpamCollection",header=None)

### 2. Pre-process messages

In [3]:
def noPunctuation(s):
    return ''.join(ch for ch in s if ch.lower() in '''abcdefghijklmnopqrstuvwxyz ''')
    
df[1] = [noPunctuation(s).lower() for s in df[1]]

### 3. Shuffle and split messages

In [4]:
np.random.seed(123)

sampling = np.random.permutation(df.shape[0])

train = df.iloc[sampling[:2500]]
validation = df.iloc[sampling[2500:3500]]
test = df.iloc[sampling[3500:]]

trainHam = train.ix[train[0]=='ham'][1].tolist()
trainSpam = train.ix[train[0]=='spam'][1].tolist()

tv = df.iloc[sampling[:3500]]
tvHam = tv.ix[tv[0]=='ham'][1].tolist()
tvSpam = tv.ix[tv[0]=='spam'][1].tolist()

validationText = validation[1]
validationLabels = validation[0]

testText = test[1]
testLabels = test[0]

### 4. Naive Bayes classifer

In [5]:
class NaiveBayesForSpam:
    def train (self, hamMessages, spamMessages):
        self.words = set(" ".join (hamMessages + spamMessages).split()) #Create a set of all words used in all texts
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)) / (len(hamMessages) + len(spamMessages)) #Probability of ham
        self.priors[1] = 1.0 - self.priors[0] #Probability of spam
        self.likelihoods = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages) #P(W|Ham)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages) #P(W|Spam)
            self.likelihoods.append([min(prob1,0.95),min(prob2,0.95)]) #Prevents probabilities from going too close to 1 for div0 errors
        self.likelihoods = np.array(self.likelihoods).T

    def train2 (self, hamMessages, spamMessages):
        self.words = set(" ".join (hamMessages + spamMessages).split()) #Create a set of all words used in all texts
        self.priors = np.zeros(2)
        self.priors[0] = float(len(hamMessages)) / (len(hamMessages) + len(spamMessages)) #Probability of ham
        self.priors[1] = 1 - self.priors[0]
        self.likelihoods = []
        spamkeywords = []
        for i, w in enumerate (self.words):
            prob1 = (1.0 + len([m for m in hamMessages if w in m])) / len(hamMessages) #P(W|Ham)
            prob2 = (1.0 + len([m for m in spamMessages if w in m])) / len(spamMessages) #P(W|Spam)
            if prob1 * 20 < prob2: #Only store words where word is 20 times more likely to appear in spam than in ham
                self.likelihoods.append([min(prob1,0.95),min(prob2,0.95)])
                spamkeywords.append(w)
        self.words = spamkeywords
        self.likelihoods = np.array(self.likelihoods).T

    def predict (self, message):
        posteriors = np.copy (self.priors)
        for i, w in enumerate (self.words):
            if w in message.lower():
                posteriors *= self.likelihoods[:,i]
            else:
                posteriors *= np.ones(2) - self.likelihoods[:,i]
            posteriors = posteriors / np.linalg.norm(posteriors) #Normalising
        if posteriors[0] > 0.5:
            return ['ham', posteriors[0]]
        return ['spam', posteriors[1]]

    def score (self,messages,labels):
        confusion = np.zeros(4).reshape(2,2)
        for m, l in zip(messages, labels):
            if self.predict(m)[0] == 'ham' and l == 'ham':
                confusion[0,0] += 1
            elif self.predict(m)[0] == 'ham' and l == 'spam':
                confusion[0,1] += 1
            elif self.predict(m)[0] == 'spam' and l == 'ham':
                confusion[1,0] += 1
            elif self.predict(m)[0] == 'spam' and l == 'spam':
                confusion[1,1] += 1
        return (confusion[0,0] + confusion[1,1]) / float (confusion.sum()), confusion

### 5. Explaining the code

$train1$ and $train2$ starts with very similar code. They first create a set of all words used in the spam and ham messages. Next, they calculate the prior probabilities $\mathbb{P}(Spam)$ and $\mathbb{P}(Ham)$. From here, the two algorithms differ.

$train1$ calculates $\mathbb{P}(W|Spam)$ and $\mathbb{P}(W|Ham)$ for all words and saves the likelihoods. It also sets a maximum on the likelihood of any word at 0.95.

$train2$ also calculates $\mathbb{P}(W|Spam)$ and $\mathbb{P}(W|Ham)$. However, it only saves the likelihoods of words where $\mathbb{P}(W|Spam) > 20 \cdot \mathbb{P}(W|Ham)$, where the liklihood of a word appearing in a Spam message is 20 times higher than the likelihood of the word appearing in a Ham message.

The $predict$ function gives the predicted label based on the posterior probability of a message being Spam or Ham. The Naive Bayes assumption is used in this step. For each word in the set of words created in the training step, if the word appears in the message, we multiply the posterior probability by the likelihood of that word in Spam and Ham. Otherwise, we multiply the posterior by (1 - likelihood) of the word in Spam and Ham. We are able to simply multiply the likelihood based on the Naive Bayes assumption of conditional independence. Finally, the posterior probability is normalised so that $\mathbb{P}(Spam|X) + \mathbb{P}(Ham|X) = 1$. Then the algorithm returns the label for the higher probability.

Finally, the $score$ function goes through every message given and calls the $predict$ function on it. It then checks the predicted label with the actual label and increases the count in the confusion matrix accordingly.

### 6. Train classifers

In [6]:
nb = NaiveBayesForSpam()
nb.train(trainHam,trainSpam)

nb2 = NaiveBayesForSpam()
nb2.train2(trainHam,trainSpam)

### 7. Performance of classifiers

In [7]:
labels = [nb.predict(text)[0] for text in validationText]
errors = np.mean(labels != validationLabels)

labels2 = [nb2.predict(text)[0] for text in validationText]
errors2 = np.mean(labels2 != validationLabels)

In [8]:
print('Error from train1 is', errors)
print('Error from train2 is', errors2)

Error from train1 is 0.041
Error from train2 is 0.03


We note that the error from train2 is significantly lower than the error from train1.

### 8. Speed and accuracy of classifiers

We expect $train2$ to be much faster than $train1$ simply because the length of saved words in $train2$ is much shorter than $train1$, since we only saved words that have a significantly higher chance of being a spam key word.

The increased accuracy is expected as well. Consider common words such as "and" or "the". These words could be very prevalent in Spam messages and would increase the posterior probability of a message being Spam. However, these words are also commonly used in Ham messages. Thus, by only considering words that occur much more often in Spam, we reduce the chance of having false positives.

### 9. False Positives

In [9]:
falsepos1 = sum([a and b for a, b in zip(pd.Series(labels) == 'spam', validationLabels == 'ham')])
falsepos2 = sum([a and b for a, b in zip(pd.Series(labels2) == 'spam', validationLabels == 'ham')])

print('There are',falsepos1, 'false positives using train1')
print('There are',falsepos2, 'false positives using train2')

There are 14 false positives using train1
There are 1 false positives using train2


A more moderate approah would be to increase the threshold to classify a message as spam. In the above algorithm, we classified a message as Spam if $\mathbb{P}(Spam) > 0.5$. We could instead increase the threshold to a higher value such as $\mathbb{P}(Spam) > 0.7$. 

We could also increase the threshold to 1, where we classify ALL messages as Ham. By simply classifying all messages as Ham, we reduce the number of false positives to 0, but at the expense of large number of false negatives. 

### 10. Confusion matrix of train2

In [10]:
nb3 = NaiveBayesForSpam()
nb3.train2(tvHam,tvSpam)

In [11]:
testError, confusion = nb3.score(testText,testLabels)

In [12]:
print(testError)
print(confusion)

0.97249034749
[[ 1800.    53.]
 [    4.   215.]]
